# Final Result Notebook

Steps : 
1. Search Full questions+title in *arq_ans_ques* index
2. Save these results as Baseline-run

----
1. Now again - Search Full questions+title in *arq_ans_ques* index
2. Rerank them using BERT - save results
3. Merge Baseline-run and Rerank-run

In [1]:
import glob
from tqdm.notebook import tqdm
import os
import jsonlines
import re
import xml.etree.ElementTree as ET
from elasticsearch import Elasticsearch
import re
from collections import defaultdict
import numpy as np
from scipy import spatial
import numpy as np
import gc
import math
import torch
from transformers import *
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

ques_path = '/data/szr207/dataset/ArqMath/jsons/questions/all.ques.jsonl'
ans_path = '/data/szr207/dataset/ArqMath/jsons/answers/all.ans.jsonl'
topic_file_path = "/data/szr207/dataset/ArqMath/Task1/Topics/Topics_V2.0.xml"


In [2]:
class Topic:
    """
    This class shows a topic for task 1. Each topic has an topic_id which is str, a title and question which
    is the question body and a list of tags.
    """

    def __init__(self, topic_id, title, question, tags):
        self.topic_id = topic_id
        self.title = title
        self.question = question
        self.lst_tags = tags


class TopicReader:
    """
    This class takes in the topic file path and read all the topics into a map. The key in this map is the topic id
    and the values are Topic which has 4 attributes: id, title, question and list of tags for each topic.

    To see each topic, use the get_topic method, which takes the topic id and return the topic in Topic object and
    you have access to the 4 attributes mentioned above.
    """

    def __init__(self, topic_file_path):
        self.__map_topics = self.__read_topics(topic_file_path)

    def __read_topics(self, topic_file_path):
        map_topics = {}
        tree = ET.parse(topic_file_path)
        root = tree.getroot()
        for child in root:
            topic_id = child.attrib['number']
            title = child[0].text
            question = child[1].text
            lst_tag = child[2].text.split(",")
            map_topics[topic_id] = Topic(topic_id, title, question, lst_tag)
        return map_topics

    def get_topic(self, topic_id):
        if topic_id in self.__map_topics:
            return self.__map_topics[topic_id]
        return None

def remove_stop(query):
    with open('englishST.txt') as f:
        all_stopwords = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    all_stopwords = [x.strip() for x in all_stopwords] 
    text_tokens = query.split(' ')
    query = [word for word in text_tokens if not word in all_stopwords]
    query = ' '.join(query)
    return query

def remove_punct(my_str):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    # To take input from the user
    # my_str = input("Enter a string: ")

    # remove punctuation from the string
    no_punct = ""
    for char in my_str:
        if char not in punctuations:
            no_punct = no_punct + char

    # display the unpunctuated string
    return no_punct

es = Elasticsearch(['http://csxindex05:9200/'], verify_certs=True)
queries = []
#"In this example, the title and the question body of topic with id A.1 is printed."
topic_reader = TopicReader(topic_file_path)
dict_q_a = defaultdict(list)

topic_list = []
with open('../runs/qrel_task1', 'r') as eval_file:
    for _,line in enumerate(eval_file):
        topic_list.append(line.split('\t')[0])

topic_list = list(set(topic_list))

list_p = []
with open('../runs/qrel_task1', 'r') as eval_file:
    for _,line in enumerate(eval_file):
        list_p.append(line.split('\t')[2])

list_p = list(set(list_p))

with open('tf.psu-task1-prim.mlt-auto-both-A.tsv', 'w') as eval_file:
    for topic_id in tqdm(topic_reader._TopicReader__map_topics):
        if topic_id in topic_list:
            title = re.sub('<[^<]+?>', '', topic_reader.get_topic(topic_id).title)
            body = topic_reader.get_topic(topic_id).question
            body_pro = re.sub('<[^<]+?>', '', body)
            query = title + '. ' + body_pro
            queries.append(query)
    #         query = query.lower()
    #         query = remove_stop(query)
            print(topic_id, query[:80], '...')
            body = {
                "size": 1000,
                 "query": {
                       "more_like_this" : {
                    "fields" : ["body"],
                    "like" : query,
                }
                }
            }

            res = es.search(index="arq_ans_ques", body=body, request_timeout=1000)

            for result in res['hits']['hits']:
                if str(result['_source']['post_id']) in list_p:
                    dict_q_a[topic_id].append(result['_source']['post_id'])

            count = 1
            for result in res['hits']['hits']:
                if str(result['_source']['post_id']) in list_p:
                    eval_file.write(topic_id+'\t'+ '1\t' +str(result['_source']['post_id'])+'\t'+str(count)+'\t'+ str(result['_score'])+'\t'+ 'mlt_base'+'\n')
                    count+=1

A.1 Finding value of $c$ such that the range of the rational function $f(x) = \frac{ ...
A.3 Approximation to $\sqrt{5}$ correct to an exactitude of $10^{-10}$. I am attempt ...
A.4 How to compute this combinatoric sum?. I have the sum  $$\sum_{k=0}^{n} \binom{n ...
A.5 A family has two children. Given that one of the children is a boy, what is the  ...
A.7 Finding out the remainder of $\frac{11^\text{10}-1}{100}$ using modulus.    If $ ...
A.8 finding value of $\lim_{n\rightarrow \infty}\sqrt[n]{\frac{(27)^n(n!)^3}{(3n)!}} ...
A.9 Simplifying this series. I need to write the series   $$\sum_{n=0}^N nx^n$$   in ...
A.10 Find the values of a>0 for which the improper integral $\int_{0}^{\infty}\frac{\ ...
A.11 What's the cross product in 2 dimensions?. The math book i'm using states that t ...
A.12 Finding the roots of a complex number. I was solving practice problems for my up ...
A.13 How to simplify expression $\int_a^b f(x)dx+\int_{f(a)}^{f(b)} f^{-1}(x)dx \ ?$. ...
A.14 Help solving

In [3]:
dict_ques = {}
dict_ans = {}
dict_aid_body = {}

with jsonlines.open(os.path.join(ques_path)) as reader:
        for obj in tqdm(reader):
            dict_ques[obj['post_id']] = obj
    
with jsonlines.open(os.path.join(ans_path)) as reader:
        for obj in tqdm(reader):
            dict_ans[obj['post_id']] = obj

for a_id in tqdm(list(dict_ans.keys())):
    ans_body = re.sub('<[^<]+?>', '',  dict_ans[a_id]['body'])
    qid = dict_ans[a_id]['parent_id']
    ques_body = re.sub('<[^<]+?>', '',  dict_ques[qid]['body'])
    ques_title = re.sub('<[^<]+?>', '',  dict_ques[qid]['title'])
    dict_aid_body[a_id] = ques_title + '. ' + ques_body #+ '. ' + ans_body

### ReRanking using fine-tuned HF RoBERTa-Base

In [4]:
from transformers import pipeline
from transformers import *

# feat_ext = pipeline("feature-extraction", model="shauryr/checkpoint-4000000", tokenizer='roberta-base', device=0) #our finetuned model

Model name 'shauryr/checkpoint-475000' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased, bart-large, bart-large-mnli, bart-large-cnn, bart-large-xsum, openai-gpt, transfo-xl-wt103, gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2, ctrl, xlnet-base-cased, xlnet-large-cased, xlm-mlm-en-2048, xlm-mlm-ende-1024, xlm-mlm-enfr-1024, xlm-mlm-enro-1024, 

In [5]:
ques_emb = []
for query in tqdm(queries):
    ques_emb.append(np.mean(feat_ext(query)[0], axis=0))

del feat_ext
torch.cuda.empty_cache()
gc.collect()

463

In [6]:
dict_q_idx = {}
count = 0
for topic_id in tqdm(topic_reader._TopicReader__map_topics):
    dict_q_idx[topic_id]=0
    count+=1

In [7]:
def Sort_Tuple(tup):  
    # reverse = None (Sorts in Ascending order)  
    # key is set to sort using second element of  
    # sublist lambda has been used  
    tup.sort(key = lambda x: x[2])
    return tup

In [8]:
def reduce_str(tokenizer, body):
    encoded = tokenizer.encode(body)[:510]
    return tokenizer.decode(encoded)

In [ ]:
result_list = []
count = 0
for qid in tqdm(topic_reader._TopicReader__map_topics):
    if qid in topic_list:
        tup_postid_sim = []
        feat_ext = pipeline("feature-extraction", model="shauryr/checkpoint-475000", tokenizer='roberta-base', device=0) #our finetuned model
        for post_id in tqdm(dict_q_a[qid]):
            try:
                feat = feat_ext(dict_aid_body[int(post_id)])[0]
                ans_emb = np.mean(feat, axis=0)
                del feat
                gc.collect()
            except:
                print(post_id)
                continue
            result = 1 - spatial.distance.cosine(ques_emb[dict_q_idx[qid]], ans_emb)

            if math.isnan(result):
                count+=1
                print(qid, post_id, count)
                continue
            tup_postid_sim.append((qid,post_id,result))
        try:
            del feat_ext
            gc.collect()
            torch.cuda.empty_cache()
        except:
            print(qid)
            continue

        result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:1000])

Model name 'shauryr/checkpoint-475000' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese, bert-base-german-cased, bert-large-uncased-whole-word-masking, bert-large-cased-whole-word-masking, bert-large-uncased-whole-word-masking-finetuned-squad, bert-large-cased-whole-word-masking-finetuned-squad, bert-base-cased-finetuned-mrpc, bert-base-german-dbmdz-cased, bert-base-german-dbmdz-uncased, bert-base-japanese, bert-base-japanese-whole-word-masking, bert-base-japanese-char, bert-base-japanese-char-whole-word-masking, bert-base-finnish-cased-v1, bert-base-finnish-uncased-v1, bert-base-dutch-cased, bart-large, bart-large-mnli, bart-large-cnn, bart-large-xsum, openai-gpt, transfo-xl-wt103, gpt2, gpt2-medium, gpt2-large, gpt2-xl, distilgpt2, ctrl, xlnet-base-cased, xlnet-large-cased, xlm-mlm-en-2048, xlm-mlm-ende-1024, xlm-mlm-enfr-1024, xlm-mlm-enro-1024, 

2916472


In [ ]:
with open('tf.psu-task1-prim.mlt.bert-auto-both-A.tsv', 'w') as eval_file:
    for res in result_list:
        count = 1
        for tuples in res:
            eval_file.write(tuples[0]+'\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'mlt_bert'+'\n')
            count+=1

In [ ]:
result_list = []

feat_ext = pipeline("feature-extraction", model="/data/szr207/github/transformers/examples/language-modeling/output/checkpoint-3000000", tokenizer='roberta-base', device=0) #our finetuned model
tokenizer =  RobertaTokenizer.from_pretrained('roberta-base')

count = 0
for qid in tqdm(list(topic_reader._TopicReader__map_topics.keys())):
    tup_postid_sim = []
    for post_id in tqdm(dict_q_a[qid]):
        try:
            feat = feat_ext(reduce_str(tokenizer, dict_aid_body[int(post_id)]))[0]
            ans_emb = np.mean(feat, axis=0)
        except:
            print('CUDA error: an illegal memory access was encountered', post_id)
            break
        result = 1 - spatial.distance.cosine(ques_emb[dict_q_idx[qid]], ans_emb)
        
        if math.isnan(result):
            count+=1
            print(qid, post_id, count)
            break
        tup_postid_sim.append((qid,post_id,result))
#     try:
#         del feat_ext
#         gc.collect()
#         torch.cuda.empty_cache()
#     except:
#         print(qid)
#         break
        
    result_list.append(Sort_Tuple(tup_postid_sim)[::-1][:1000])

In [ ]:
# del feat_ext
gc.collect()
torch.cuda.empty_cache()

In [ ]:
with open('tf.psu-task1-mlt.bert-auto-both-A.tsv', 'w') as eval_file:
    for res in result_list:
        count = 1
        for tuples in res:
#             eval_file.write(tuples[0]+'\t'+ '1\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'mlt_bert'+'\n')
            eval_file.write(tuples[0]+'\t' + str(tuples[1]) +'\t'+str(count)+'\t'+ str(tuples[2])+'\t'+ 'mlt_bert'+'\n')
            count+=1

#### Merging BERT and TF-IDF using RRF

In [ ]:
from trectools import TrecRun, TrecEval, fusion

runs_path = '/data/szr207/projects/ArqMath/'
r1 = TrecRun(os.path.join(runs_path, "tf.psu-task1-mlt.bert-auto-both-A.tsv"))
r2 = TrecRun(os.path.join(runs_path, "tf.psu-task1-mlt.base-auto-both-A.tsv"))

# Easy way to create new baselines by fusing existing runs:
fused_run = fusion.reciprocal_rank_fusion([r1,r2])

# Save run to disk with all its topics
fused_run.print_subset("tf.psu-task1-rrf.base.bert-auto-both-P.tsv", topics=fused_run.topics())

In [ ]:
df = pd.read_csv('tf.psu-task1-rrf.base.bert-auto-both-P.tsv', header=None , sep=" ")
df = df.drop(columns=1)
df.to_csv('psu-task1-rrf.base.bert-auto-both-P.tsv', sep='\t', index=False, header=False)

In [ ]:
!java -jar jtreceval-0.0.5-jar-with-dependencies.jar -m ndcg /data/szr207/dataset/ArqMath/Task1/Sample\ Topics/qrel.V1.0.tsv es.mlt.final.finetune.dat

In [1]:
!nvidia-smi

Sun Jul  5 23:43:11 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce RTX 208...  Off  | 00000000:18:00.0 Off |                  N/A |
| 27%   32C    P8    11W / 250W |   3097MiB / 11019MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 208...  Off  | 00000000:3B:00.0 Off |                  N/A |
| 49%   58C    P2   233W / 250W |   1438MiB / 11019MiB |     65%      Default |
+-------------------------------+----------------------+----------------------+
|   2  G